<h1 align='center'> Fast and Accurate Recurrent Neural Network Acoustic Models for Speech Recognition 🗣️ </h1>

<h2 align='center'> - </h2>

<h3 align='center'> Techniques that further improve performance of LSTM RNN acoustic models for large vocabulary speech recognition </h3>

This notebook is directly inspired by the paper [Fast and Accurate Recurrent Neural Network Acoustic Models for Speech Recognition](https://arxiv.org/abs/1507.06947) released in July 2015. The latter will be refered as 'the paper' during the whole notebook.

<br>

## **Introduction to Speech Recognition**  
Speech Recognition, or Automatic Speech Recognition (**ASR**), refers to the process of converting spoken language into text using computational models. Unlike **voice recognition**, which aims to identify the speaker based on their unique vocal characteristics, or **emotion recognition**, which detects the emotional state of a speaker, ASR focuses purely on transcribing the linguistic content of speech. Modern ASR systems leverage deep learning models, particularly Recurrent Neural Networks (RNNs) and Transformer-based architectures, to achieve high accuracy in various acoustic environments.

Over time, several methods have been developed to tackle speech recognition, such as:  
- Gaussian Mixture Models (GMMs),  
- Hidden Markov Models (HMMs),  
- Deep Neural Networks (DNNs).  

However, because speech is a naturally occurring time sequence, it requires a neural network capable of processing sequential data effectively. Additionally, the system must be lightweight in terms of memory and computational requirements, as the goal is to achieve real-time speech recognition. **Recurrent Neural Networks (RNNs), particularly Long Short-Term Memory (LSTM) networks, are well-suited for this task.** 

## **I. Representing speech signals**  
A speech signal is a continuous waveform that conveys information through variations in air pressure.   
Let's take an example:

In [1]:
import soundfile as sf                                          

path = 'data/audio_sample/84-121123-0000.flac'                                                  
waveform, sample_rate = sf.read(path)  

In [ ]:
from IPython.display import Audio

audio = Audio(waveform, rate=sample_rate)
display(audio)

#### To process it computationally, there are several meaningful representations:

1. **Raw waveform**: The most basic form of representation.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(22, 8))
plt.plot(waveform)
plt.title("Audio signal waveform")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.grid()
plt.show()

Unfortunately, it is not directly suitable for ASR due to:
- **High dimensionality**: A typical speech signal sampled at 16 kHz produces 16,000 values per second, making direct processing computationally expensive.  
- **Speaker and noise variability**: Raw waveforms are highly sensitive to variations in speaking style, microphone quality, and background noise... 
- **Lack of structured information**: ASR models struggle to extract phonetic and linguistic patterns directly from waveforms due to the complexity of raw acoustic signals. 

<br>

2. **Fourier transform**: Converts the signal into a sum of sinusoidal components, revealing spectral content over time.


In [ ]:
import numpy as np

n = len(waveform)
T = 1.0 / sample_rate
yf = np.fft.fft(waveform)
xf = np.fft.fftfreq(n, T)[:n//2]

amplitude = 2.0/n * np.abs(yf[:n//2])

plt.figure(figsize=(22, 8))
plt.plot(xf, amplitude)
plt.title('Audio signal FFT')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.grid()
plt.show()

FFT is also not very suitable for ASR because:   
- **Loss of temporal information**: The FFT gives a static frequency spectrum but does not show how frequencies change over time, which is crucial for speech understanding.  
- **Too much unnecessary detail**: The FFT produces a very detailed frequency representation, but not all frequencies contribute equally to speech perception. The human ear, for instance, perceives frequencies on a non-linear scale (Mel scale).  
- **No emphasis on speech-specific features**: It does not highlight the phonetic structures and speech-relevant frequency bands needed for ASR models. 

<br>

3. **Spectrogram**: A time-frequency representation showing how signal energy is distributed across different frequencies over time.

In [ ]:
import librosa
import librosa.display

D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform)), ref=np.max)
plt.figure(figsize=(22, 8))
librosa.display.specshow(D, sr=sample_rate, x_axis='time', y_axis='log', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title('Audio signal spectrogram (dB)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

The spectogram will not be used for ASR because:
- **High redundancy**: Spectrograms contain a lot of information that is not necessary for recognizing phonemes, such as harmonics unrelated to speech content.  
- **Linear frequency scale**: Standard spectrograms use a linear frequency scale, while human perception is more logarithmic, making them less efficient for speech feature extraction.  
- **Sensitivity to noise and variability**: While spectrograms capture useful phonetic structures, they can be affected by background noise and variations in pronunciation, making direct usage less optimal.  

<br>

4. **Mel spectrogram**: A spectrogram processed with the Mel scale, which mimics human auditory perception by emphasizing perceptually relevant frequency bands.

In [ ]:
mel_spectrogram = librosa.feature.melspectrogram(y=waveform, sr=sample_rate, n_mels=128)
mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

plt.figure(figsize=(22, 8))
librosa.display.specshow(mel_spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title('Audio signal Mel spectrogram (dB)')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()


**Mel spectrogram** is a better alternative to spectrogram because it applies a mel scale transformation to mimic human auditory perception. The mel scale is a **perceptual** scale of pitches judged by listeners to be equal in distance from one another. A formula ([O'Shaughnessy](https://en.wikipedia.org/wiki/Daniel_O%27Shaughnessy) 1987) to convert f (Hz) into m (mels) is:

$$ m = 2595 \times log_{10}(1 + \frac{f}{700})$$



Looking at the spectrogram and the mel spectrogram, it is remarkable that the mel spectrogram contains **less information** than the classic spectrogram, because it is designed to mimic human perception rather than capturing all the raw frequencies. Thus, high frequencies are compressed and the number of frequency bands is reduced.



That is why Mel-Frequency Cepstral Coefficients (MFCCs) are widely used as input features for ASR models. They are derived from the Mel spectrogram and capture both spectral and cepstral (rate of spectral change) characteristics of speech. The main reasons for using MFCCs in ASR are:

- **Auditory inspiration**: MFCCs approximate the non-linear frequency perception of the human ear, **focusing on lower frequencies where speech information is most concentrated.**
- **Dimensionality reduction**: Instead of using raw waveforms or high-dimensional spectrograms, MFCCs provide a compact yet informative representation of speech.
- **Robustness**: They enhance ASR performance by filtering out non-linguistic variations and making the system more invariant to speaker differences and environmental noise.

More information about Mel spectrogram [here](https://importchris.medium.com/how-to-create-understand-mel-spectrograms-ff7634991056).


<br>


#### Thus, in this notebook, the inputs will be the MFCCs extracted from short audio signals and the outputs the corresponding transcripts for each audio signal.  The dataset was taken from the [Mozilla Common Voice dataset](https://commonvoice.mozilla.org/en/datasets).    


<h3 align='center'>
    <img src="img/Common_Voice_Banner2.png">
</h3>


## **II. Features processing**  

### **1. Data augmentation**
Before extracting the MFCCs, it is required to do some *data augmentation* to improve the model. In fact, audio data augmentation is the process of applying transformations to audio data to artificially increase the size and diversity of a dataset. This helps improve the **robustness** and **generalization**.    

Here are some widely used augmentation methods:

| **Technique**        | **Description** | **Effect on Model** |
|---------------------|----------------|---------------------|
| **Add Noise**        | Add white noise, background noise, or other disturbances | Helps model generalize to noisy environments |
| **Time Stretching**  | Speeds up or slows down the audio without changing pitch | Mimics variations in speech rate |
| **Pitch Shifting**   | Shifts the pitch up or down | Simulates different voices or instruments |
| **Time Shifting**    | Moves the audio forward or backward in time | Adds temporal variations |
| **Volume Scaling**   | Increases or decreases loudness | Makes the model robust to volume changes |
| **Reverberation**    | Adds echo or room effects | Simulates different acoustic environments |
| **Bandpass Filtering** | Removes or enhances specific frequency ranges | Can simulate microphone effects |   

In the paper, data augmentation was also performed on input features:
>To achieve robustness to background noise and reverberant environments we synthetically distort each utterance in a room simulator with a virtual noise source. Noise is taken from the audio of [YouTube](https://www.youtube.com/) videos. Each utterance is randomly distorted to get 20 variations. This “Multi-style training” also alleviates overfitting.

In [13]:
def add_noise(audio, noise_level=0.005):
    noise = np.random.randn(len(audio))
    return np.clip(audio + noise_level * noise, -1.0, 1.0)

def shift_audio(audio, shift_max=0.2):
    shift = int(np.random.uniform(-shift_max, shift_max) * len(audio))
    return np.roll(audio, shift)

def time_stretch(audio, rate=1.0):
    return librosa.effects.time_stretch(audio, rate=rate)

def pitch_shift(audio, sr, n_steps_range=(-3, 3)):
    n_steps = np.random.randint(n_steps_range[0], n_steps_range[1] + 1)
    return librosa.effects.pitch_shift(y=audio, sr=sr, n_steps=n_steps)

def apply_augmentations(audio, sr, noise_prob=0.8, shift_prob=0.5, stretch_prob=0.5, pitch_prob=0.5):
    
    if np.random.rand() < noise_prob:
        audio = add_noise(audio)
    
    if np.random.rand() < shift_prob:
        audio = shift_audio(audio)
    
    if np.random.rand() < stretch_prob:
        audio = time_stretch(audio)
    
    if np.random.rand() < pitch_prob:
        audio = pitch_shift(audio, sr)

    return audio

In [ ]:
augmented_audio = apply_augmentations(waveform, sample_rate)


plt.figure(figsize=(22, 8))
plt.subplot(1, 2, 1)
librosa.display.waveshow(waveform, sr=sample_rate)
plt.title("Original audio waveform")

plt.subplot(1, 2, 2)
librosa.display.waveshow(augmented_audio, sr=sample_rate)
plt.title("Augmented audio waveform")

plt.show()

### **2. Extracting MFCCs from audio signals**  

The extraction of the **Mel-Frequency Cepstral Coefficients (MFCCs)** from audio is described in the paper:   
>We use 80-dimensional log mel filterbank energy features computed every 10ms on 25ms windows.


<br>

Here are the detailed steps for the extraction from the audio signal $x(t)$:

#### **a) Framing the Signal**  
- The signal $x(t)$ is sampled at a rate $ f_s = 16 \text{ kHz}$, resulting in the discrete-time signal $ x[n] = x(nT_s) $, where $ T_s = \frac{1}{f_s} $ is the sampling period.  
- The signal is divided into **25ms** frames with a **10ms** stride. The corresponding number of samples per frame and hop size are given by:  
  $$N = \text{round}(25 \times 10^{-3} \cdot f_s)$$
  $$H = \text{round}(10 \times 10^{-3} \cdot f_s)$$ 
- A window function $ w(n) $ (e.g., [**Hamming window**](https://en.wikipedia.org/wiki/Window_function#Hann_and_Hamming_windows)) is applied to minimize discontinuities at the frame edges.  
- The windowed signal for frame $ m $ is given by:  
  $$x_w[m, n] = x[n + mH] \cdot w[n], \quad 0 \leq n < N, \quad m = 0, 1, 2, \dots$$ 

---

#### **b) Fourier Transform (FFT)**  
The **Fast Fourier Transform (FFT)** is applied to obtain the frequency spectrum $X(f)$.  
  $$X(f) = \sum_{n=0}^{N-1} x_w(n) e^{-j 2\pi f n / N}$$

where, $f_n = \frac{n f_s}{N}, \quad n = 0, 1, ..., N-1$


The **power spectrum**, $P(f)$, represents the energy at each frequency bin $f$ and can be computed as: $$P(f) = |X(f)|^2$$

---

#### **c) Applying the Mel filterbank**  
The **Mel filter bank** chosen consists of a set of overlapping **80 triangular filters** $H_m(f)$, designed to mimic human auditory perception. Each filter emphasizes a specific range of frequencies in the **Mel scale**.

The **filtered energy** for the $m$-th Mel band is computed as:

$$S_{\text{mel}}(m) = \sum_{f=0}^{F-1} H_m(f) P(f)$$

where:
- $H_m(f)$ is the weight of the **m-th Mel filter** at frequency bin $f$, forming a set of overlapping triangular filters.
- $P(f)$ is the **power spectrum** obtained from the FFT.
- $S_{\text{mel}}(m)$ is the **filtered energy** corresponding to the $m$-th Mel frequency band.


<h3 align='center'>
    <img src="img/mel_bank_filter.png">
</h3>



#### **d) Logarithm of filtered energy**  
Since human perception of loudness is logarithmic, a **logarithm** is applied to the Mel-scaled spectrum:

$$E_m = \log S_{\text{mel}}(m) = \log \sum_{f=0}^{F-1} H_m(f) P(f)$$

where:
- $E_m$ is the **logarithmic Mel energy**.
- $H_m(f)$ is the response of the $m$-th Mel filter.

#### **e) Discrete Cosine Transform (DCT)**  
Finally, a **Discrete Cosine Transform (DCT)** is applied to the **log Mel energy vector** to obtain the **Mel-Frequency Cepstral Coefficients (MFCCs)**:

$$C_n = \sum_{m=0}^{M-1} E_m \cos \left( \frac{\pi n (2m+1)}{2M} \right)$$

where:
- $C_n$ is the $n$-th **MFCC coefficient**.
- $M$ is the total number of Mel filter banks.

<br>

The [librosa](https://librosa.org/doc/latest/index.html) library computes the extraction easily!   
*Get more information about MFCCs in this [paper](https://ieeexplore.ieee.org/document/9955539).*

In [15]:
def extract_features(audio, n_mels=80, frame_length=0.025, frame_stride=0.01, sr=16000):
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
    log_mel = librosa.power_to_db(mel_spec)
    return log_mel.T  # (time, features)

Then, a frame 'stacking' and 'decimation' is performed as described in the paper:

>One way of reducing the huge space of alignments is to reduce the number of input frames. This can be done by simply decimating the input frames, though to present the full acoustic information of the input signal, we first stack frames so that the networks sees multiple (e.g. 8) frames at a time but then decimate the frames so that we skip forward multiple frames (e.g. 3) after processing each such “super-frame”. [...]   
By decimating the frames in this manner, the acoustic model is able to process the full signal but acoustic model computation need only happen every 30ms. For a network of a fixed size this results in a dramatic reduction in the acoustic model computation and decoding time.

<br>

It is useful for ASR because:

- **Computational efficiency**: By reducing the number of frames processed, the acoustic model runs faster, reducing both **training time** and **real-time inference cost**.

- **Maintaining full acoustic information**: Even though we skip frames, stacking multiple frames before skipping ensures that the model still has access to a **rich temporal context**.  

- **Better generalization**: Since ASR models rely on capturing phonetic patterns over time, stacking frames allows them to recognize speech structures more effectively, rather than making predictions based on a single, noisy frame.  


<h3 align='center'>
    <img src="img/stacking.png">
</h3>

In [16]:
def stack_decimate_frames(features, stack_size=3, skip=3):
    stacked_features = []
    for i in range(0, len(features) - stack_size + 1, skip):
        stacked_features.append(np.concatenate(features[i:i+stack_size], axis=0))
    return np.array(stacked_features)

#### Let's make an example of augmentation, MFCCs extraction and stacking and decimating:

In [ ]:
augmented_audio = apply_augmentations(waveform, sample_rate)
extracted_features = extract_features(augmented_audio)
print(f"extracted_features:\n{extracted_features}")
print(f"size extracted_features:{extracted_features.shape}\n")

stacked_extracted_features = stack_decimate_frames(extracted_features)
print(f"stacked_extracted_features:\n{stacked_extracted_features}")
print(f"size stacked_extracted_features:{stacked_extracted_features.shape}\n")

#### In this example, the feature output size is **(22, 240)**:

#### **22: Number of frames after decimation**   
The original audio produced $N = 66$ **MFCCs frames** before stacking (dimension 0 of the `extracted_features` vector). After **decimation**, with `skip=3`, the number of remaining frames becomes **N' ≈ N / 3**. The **first dimension** (22) represents then the final number of **super-frames** processed after stacking and decimation.


---

#### **240: Size of the feature vector**      
Before stacking, each frame had **80 MFCC coefficients** (because of the bank of 80 Mel filters). With `stack_size=3`, **3 frames** are stacked to form a **super-frame**. The concatenation of the MFCC vectors gives $80 \times 3 = 240$. The **second dimension** 240 corresponds to the **length of the feature vector per super-frame**.


*Note: The first dimension can vary, depending on the length of the audio, while the second one will always be 240 in this case.* 

### **3. Encoding audio transcripts**   
Now that the input data is processed, it is necessary to take care of the target data: the **audio transcripts**. An audio transcript is the written representation of the spoken content in an audio signal.    
It is not possible to pass the sentences themselves as targets. Thus, the question of how to represent the spoken content raises: Represent the transcript **letter by letter**, **word by word**, or **phonematically** (using phonemes)?

<br>

- #### **Letter-by-letter representation**  
    Representing speech transcription letter by letter is not ideal for several reasons:
    - **Context dependency**: Letters in words can have different pronunciations based on their context. For example, the letter "c" can be pronounced as /k/ in "cat" but as /s/ in "cent." This variation makes it challenging to model letters directly in speech recognition.
    - **Inefficiency**: Since the ASR model would need to account for all possible variations of each letter in different contexts, this representation would make the training process more complex and less efficient.   
    *Note: Some well-known and effective models are based on predicting letters. (cf some of these [torch models](https://pytorch.org/audio/main/models.html))*   

<br>

- #### **Word-by-word representation**  
    Representing transcripts word by word also presents challenges:
    - **Vocabulary size**: A word-level representation would require a very large dictionary to cover the many thousands of words in any natural language. This can lead to an enormous model size and complicate the recognition process.
    - **Out-of-vocabulary words**: ASR models would struggle with new words or names that are not part of the fixed vocabulary, resulting in inaccurate or failed transcriptions.
  
<br>

- #### **Phonetic representation**  
    The most effective approach in ASR is to represent the transcript **phonematically**, using a **phoneme-based** transcription. Phonemes are the smallest distinct units of sound in a language. In English, for example, the word "cat" can be broken down into its phonemes: /k/, /æ/, and /t/. This approach offers several advantages:
    - **Pronunciation variability**: Phonemes capture the actual sounds of speech, irrespective of the letters used to spell the words. This allows the model to account for how words are pronounced differently in various contexts.
    - **Compact and flexible**: Phoneme-based systems reduce the vocabulary size since they only need to recognize a set of phonemes (e.g., around 40 phonemes in English), as opposed to the vast number of possible words. This makes the ASR model more manageable and flexible.
    - **Language agnostic**: Phonemes are language-independent and can be used for any spoken language, allowing the system to generalize across accents and dialects.

<br>

So now the next steps are to:   
- Convert all the transcripts into their phoneme sequences
- Build a dictionary of phonemes
- Encode each phoneme as an integer (for the model)


#### a) Convert all the transcripts into their phoneme sequences
This will be realized by the [g2p](https://github.com/Kyubyong/g2p) library, which is a module for English grapheme to phoneme conversion:

In [18]:
import re
from g2p_en import G2p
g2p = G2p()


def text_to_phonemes(text):
    # Clean up text: remove punctuation
    text = re.sub(r"[^\w\s]", "", text)

    phonemes = g2p(text)

    # Take the first pronunciation and remove the numbers indicating stress (e.g. AH1 → AH)
    phonemes = [p.rstrip('012') for p in phonemes]

    # Replace the blank space ' ' → '-'
    phonemes = ['-' if item == ' ' else item for item in phonemes]
    return phonemes

#### b) Build a dictionary of phonemes

In [19]:
# List of authorized ARPAbet phonemes (40 English phonemes)
PHONEMES = [
    "IY", "IH", "EH", "AE", "AA", "AH", "AO", "UH", "UW", "ER", "AX", "EY", "AY", "OW", "OY", "AW",
    "P", "B", "T", "D", "K", "G", "CH", "JH", "F", "V", "TH", "DH", "S", "Z", "SH", "ZH",
    "HH", "M", "N", "NG", "L", "R", "W", "Y"
]

#### c) Encode each phoneme as an integer (for the model)

In [20]:
# Vocabulary building (phoneme mapping → index)
vocab = {phoneme: (idx + 2) for idx, phoneme in enumerate(PHONEMES)}

# Adding blank space '-'
vocab['-'] = 1


def encode_phonemes(text, vocab):
    phoneme_list = text_to_phonemes(text)
    encoded_sequence = [vocab.get(p) for p in phoneme_list]
    
    return encoded_sequence

In [ ]:
vocab

In the paper, another target is defined:
>The targets [...] can be efficiently and easily computed using **finite state transducers** (FSTs) [...] with additional optional blank states interposed between the states of the sequence labels.

A **Finite-State Transducer (FST)** is a type of **finite-state machine** that maps input sequences to output sequences. In **speech recognition**, FSTs are widely used to model the relationships between different levels of linguistic representation:

<h3 align='center'>
    <img src="img/fst.jpeg">
</h3>

FSTs will not be used in this notebook as it will increase the complexity, but you can find more information [here](https://ieeexplore.ieee.org/document/7178778).


<br>

Let's now take an example with a "difficult" sentence:   

<h4 align='center'>
    'Tymoshenko won the gold medal in all around ahead of Skaldina.'
</h4>

In [ ]:
transcript = "Tymoshenko won the gold medal in all around ahead of Skaldina."
encoded_phonemes = encode_phonemes(transcript, vocab)
print(f"Transcript: {transcript}")
print(f"Phonemes: {text_to_phonemes(transcript)}")
print(f"Encoded phonemes: {encoded_phonemes}")

#### Now, the input data (MFCC features) and targets (encoded phonemes) are defined.   

However, some processing is still required to manage the total volume of data.   
First, the input data must be **normalized**:

In [23]:
def normalize_mfccs(mfccs):
    mean = np.mean(mfccs, axis=0, keepdims=True)
    std = np.std(mfccs, axis=0, keepdims=True)

    normalized_mfccs = (mfccs - mean) / (std + 1e-6)

    return normalized_mfccs

Then a function that handles the extraction of input data and the encoding of targets was defined:

In [24]:
def process_audio_and_text(entry, vocab, X, Y):
    # Load audio features
    y, sr = librosa.load(entry["key"], sr=16000)

    # If no features, ignore this entry
    if y is None:
        print(f"Skipping entry due to missing or invalid audio file: {entry['key']}")
        return # Exit the function for this `entry`

    # Aplly random data augmentation
    augmented_audio = apply_augmentations(y, sr)

    # Extract audio features
    features = extract_features(augmented_audio)

    # Stack and downsample
    features = stack_decimate_frames(features)
    features = normalize_mfccs(features)

    # Convert text to indices
    encoded_phonemes = encode_phonemes(entry["text"], vocab)


    # The number of audio feature frames must be greater than or equal to the number of phoneme labels...
    if (features.shape[0]) < (len(encoded_phonemes)):
        print(f"Skipping entry otherwise CTC will be infinite...")
        return 
    else:
        X.append(features)
        Y.append(encoded_phonemes)

The following code processes all the data in the training dataset and stores the normalised MFCCs features in the X_train variable and the associated transcripts in the y_train variable.   

This code takes a few minutes to run, so it will not be run here, X_train and y_train have been saved. 


```python
    import json

    X_train = []
    y_train = []

    train_file_path = 'data/train_validated.json'
    with open(train_file_path, "r", encoding="utf-8") as f:
    train_data = [json.loads(line) for line in f]

    # Apply preprocess on each sample
    for entry in train_data:
    process_audio_and_text(entry, vocab, X_train, y_train)

    # Adding padding to standardize sizes
    from tensorflow.keras.preprocessing.sequence import pad_sequences

    X_train_padded = pad_sequences(X_train, padding='post', value=0.0, dtype='float16')
    y_train_padded = pad_sequences(y_train, padding='post', value=99, dtype='int32')
    X_train = np.array(X_train_padded)
    y_train = np.array(y_train_padded)

    np.save('data/train/X_train.npy', X_train)
    np.save('data/train/y_train.npy', y_train)
```

<br>

Audio recordings and their corresponding transcripts vary in length. However, deep learning models typically require fixed-size inputs for efficient batch processing. To accommodate this, **padding** was applied to both inputs and targets, adding extra tokens (usually a special padding symbol) to the end of shorter sequences so that all sequences in a batch have the same length.   
*Note: For the input the padding value is 0.0, which corresponds to no signal and for the targets the value 99 was chosen.*

In [25]:
vocab['<PAD>'] = 99

In [26]:
X_train = np.load('data/train/X_train_augmented.npy')
y_train = np.load('data/train/y_train_augmented.npy')

In [ ]:
print("Shape of X_train:", X_train.shape)  # (num_samples, max_time_steps, n_features)
print("Shape of y_train:", y_train.shape)  # (num_samples, max_seq_length)

print("Sample X_train:", X_train[0][:5])  # Displaying the first frames
print("Sample y_train:", y_train[0])  # Displaying the encoded transcript

<div class="alert alert-info" role="alert", align="center">

**Exercise:** Can you write a function that decodes the the encoded transcript `y_train[0]` into phonemes / text ?
</div>

The same code is applied for the test dataset:

```python
    X_test = []
    y_test = []

    test_file_path = 'data/test_validated.json'

    with open(test_file_path, "r", encoding="utf-8") as f:
        test_data = [json.loads(line) for line in f]

    for entry in test_data:
        process_audio_and_text(entry, vocab, X_test, y_test)


    X_test_padded = pad_sequences(X_test, padding='post', maxlen=X_train.shape[1], value=0.0, dtype='float16')
    y_test_padded = pad_sequences(y_test, padding='post', maxlen=y_train.shape[1], value=99, dtype='int32')
    X_test = np.array(X_test_padded)
    y_test = np.array(y_test_padded)


    np.save('data/test/X_test.npy', X_test)
    np.save('data/test/y_test.npy', y_test)
```

In [28]:
X_test = np.load('data/test/X_test_augmented.npy')
y_test = np.load('data/test/y_test_augmented.npy')

## III. Model definition


Now that the inputs (**MFCC features**) and targets (encoded transcripts) are processed, let's proceed with constructing the ASR model. As described in the paper, the model is a **5-layer LSTM-based Recurrent Neural Network (RNN)**. Each LSTM layer is **bidirectional**, meaning it processes the input sequence in both forward and backward directions, capturing long-range dependencies in speech. Additionally, each layer is connected to both the previous **forward and backward layers**, ensuring effective information flow across time steps. Finally, the output layer receives connections from both the final forward and backward LSTM layers, allowing the model to generate accurate predictions by leveraging contextual information from both past and future frames. Each LSTM layer consists of **300 neurons (hidden units)** per direction. This means that at each time step, both the forward and backward LSTMs maintain a **300-dimensional hidden state**, resulting in a total hidden representation of **600 dimensions** per layer after concatenation.   
This architecture is well-suited for ASR, as it enables the model to capture phonetic structures while maintaining temporal consistency in speech transcription:

<h3 align='center'>
    <img src="img/model.png">
</h3>


For a **bidirectional LSTM**, both forward and backward contexts are incorporated by using two hidden states:  
- $\overrightarrow{h_t}$ (forward LSTM processing from left to right)
- $\overleftarrow{h_t}$ (backward LSTM processing from right to left)

Each hidden state is computed as:

$$\overrightarrow{h_t} = f(W_x \overrightarrow{x_t} + W_h \overrightarrow{h_{t-1}} + b)$$

$$\overleftarrow{h_t} = f(W_x \overleftarrow{x_t} + W_h \overleftarrow{h_{t+1}} + b)$$

The final hidden representation at each time step is obtained by concatenating the forward and backward hidden states:

$$h_t = [\overrightarrow{h_t} ; \overleftarrow{h_t}]$$

The output layer receives this combined hidden state and estimates the label posteriors as:

$$p(l_t | x_t, h_t) = p(l_t | x_t, \overrightarrow{h_t}, \overleftarrow{h_t})$$


where: 
- $x_t$ is the input vector at time step $t$.
- $f$ is the activation function (usually a combination of sigmoid and tanh).
- $W_x$ is the input weight matrix, that transforms the input $x_t$ into the LSTM's hidden space.   
- $W_h$ is the recurrent weight matrix, that determines how the hidden state from the previous time step ($h_{t-1}$) influences the current hidden state.  
- $b$ is a bias vector that allows the model to learn an offset for the activations.  
- $l_t$ is the posterior probability of the label given the input sequence and the hidden state.


*Note: The input dimension is 240 (the size of the feature vector seen earlier) and the output dimension is 42, the size of the vocabulary.*

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [30]:
import torch.nn as nn
import torch.nn.functional as F

input_dim = 240
hidden_dim = 300
num_layers = 5
num_classes = len(vocab) + 1
bidirectional = True
dropout=0.5
     

class SpeechRecognitionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout=dropout):
        super(SpeechRecognitionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.init_weights()

    # Weight initialization
    # def init_weights(self):
    #     for name, param in self.named_parameters():
    #         if 'weight' in name:
    #             nn.init.uniform_(param, -0.04, 0.04)
    
    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

    def init_hidden(self, batch_size):
        num_directions = 2 if self.lstm.bidirectional else 1
        return (torch.zeros(num_layers * num_directions, batch_size, hidden_dim),
                torch.zeros(num_layers * num_directions, batch_size, hidden_dim))

    def forward(self, x, input_lengths):
        packed_input = nn.utils.rnn.pack_padded_sequence(x, input_lengths.to(device), batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        # Memory cell activation limited to [-50, 50].
        hn = torch.clamp(hn, -50, 50)
        cn = torch.clamp(cn, -50, 50)


        output = self.fc(output)
        output = F.log_softmax(output, dim=2)
        return output


model = SpeechRecognitionModel(input_dim, hidden_dim, num_layers, num_classes)

In the paper, it is explicitly written how they initialized the model's weights:      
>The weights in all the networks are randomly initialized with a uniform (-0.04, 0.04) distribution.    

This proved to be a very poor initialization for the model, as it always converged to empty outputs. The input tensors were then filled with values using a [Xavier uniform distribution](https://pytorch.org/docs/stable/nn.init.html).

<br>


In the paper, it is also wrritten that:   
>We clip the activations of memory cells to [-50,50].

<br>

In addition, a mask has been added to ignore the 0 padding contained in the entries, thanks to the function [pack_padded_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html).   

<br>

Let's try to train the model with a single audio, here its the transcript: 

<h4 align='center'>
    'Jesus loves me, he who died.'
</h4>

In [31]:
audio_test = 'data/audio_sample/common_voice_en_39177958.mp3' 
transcript = 'Jesus loves me, he who died.'

# Extract input features
y, sr = librosa.load(audio_test, sr=16000)
features = normalize_mfccs(stack_decimate_frames(extract_features(y)))
input = torch.as_tensor(features, dtype=torch.float32).unsqueeze(0)
input_length = torch.tensor([input.shape[1]], dtype=torch.long)


# Encode target
target = np.array(encode_phonemes(transcript, vocab))
target_length = torch.tensor([len(target)], dtype=torch.long)
target = torch.tensor(target, dtype=torch.int32).unsqueeze(0)

In [ ]:
print(f"input: {input}\n")
print(f"target: {target}\n")

In [ ]:
output = model(input, input_length)
prediction = torch.argmax(output, dim=-1).squeeze(0).detach().cpu().numpy()

print(f"output: {output}\n")
print(f"prediction: {prediction}\n")

The output of the model is a sequence of **probability distributions** over the possible phonemes in `vocab` **for each timestep**.  

### **Formally:**  
- Let $T$ be the number of timesteps in the model’s output (depending on the audio duration and stride).  
- Let $N$ be the number of options in the dictionary.
- At each timestep $t$ of the processed audio, the model predicts a probability distribution $P(l | x_t)$ over all possible phonemes $l$.  

This means that for **each timestep $t$**, the model gives a **vector of size** $N$ containing the probabilities for **each possible phoneme**:

$$P(l | x_t) = \begin{bmatrix} 
P(l_1 | x_t) \\ 
P(l_2 | x_t) \\ 
\vdots \\ 
P(l_N | x_t)
\end{bmatrix}$$


In practice, many models use **log-softmax** to output the **log-probabilities** instead of raw probabilities. This is the case for this model. The log-softmax transformation applies the logarithm to the output of the softmax function:
$$\text{LogSoftmax}(z_{t,i}) = \log\left(\frac{e^{z_{t,i}}}{\sum_{j=1}^{N} e^{z_{t,j}}}\right) = z_{t,i} - \log\left(\sum_{j=1}^{N} e^{z_{t,j}}\right)$$

It is more often used for:
- **Numerical stability**: Working with log-likelihoods (log-probabilities) helps avoid numerical issues with very small probability values. Directly computing probabilities (especially when values are close to 0) can lead to underflow, whereas log-probabilities prevent this.
- **Efficiency in loss calculation**: Loss functions are typically more stable and efficient when they operate on log-likelihoods instead of raw probabilities.

In [ ]:
ce_loss = nn.CrossEntropyLoss()
ce_loss(output, target)

<details class="alert alert-danger">
    <summary markdown="span"><b>An error occured. Can you guess why? (click to expand)</b></summary>


In ASR, the model’s output is based on **time-dependent** audio features, whereas the target is a **textual representation** of spoken words. The model produces a sequence of predictions at a high frame rate (e.g., every 30ms), while the target transcript consists of a much shorter sequence of phonemes. So, in most cases he length of the model's output sequence does not match the length of the target transcript. It is thus **not possible to directly apply cross-entropy loss** at each time step, as there is no one-to-one alignment between the model’s predictions and the ground truth labels. Instead, the model must learn how to **map variable-length sequences of audio frames to a much shorter sequence of text**: thanks to a specialized loss function, such as **Connectionist Temporal Classification (CTC) loss**, which allows the model to learn an optimal alignment between input frames and target sequences without requiring explicit per-frame supervision.
</details>

In [35]:
ctc_loss = nn.CTCLoss(blank=0, reduction='mean')

In [ ]:
loss = ctc_loss(
    output.permute(1, 0, 2),
    target,
    input_length,
    target_length
)

print(f'Loss: {loss.item()}')

The **CTC** loss is a complex but very interesting mathematically tool defined to handle the alignment problem in ASR:

#### **1. Probability of an alignment $P(\pi | x)$**  
Given an input sequence of length $T$ and a target transcript of length $U$ , the ASR model produces a probability distribution over possible output symbols (including a special blank symbol $\varnothing$) at each time step.    
The blank token ($\varnothing$)  is a special symbol used by the CTC loss function to represent "no output" at a given timestep. This allows the model to "skip" certain time steps when the network doesn't need to output a phoneme or symbol. 


For a given alignment $\pi = (\pi_1, \pi_2, ..., \pi_T)$, the probability of this alignment given the input $x$ is:  

$$P(\pi | x) = \prod_{t=1}^{T} P(\pi_t | x_t)$$

where $P(\pi_t | x_t)$ is the probability of emitting symbol $\pi_t$ at time step $t$, predicted by the model.

---

#### **2. Marginalizing over all possible alignments**  
The key idea of CTC is that multiple alignments $\pi$ can correspond to the same final target transcript $y$.   
For example, the word "hello" can be stretched across multiple time steps in different ways:  

- $h\varnothing e\varnothing l\varnothing l o$  
- $hhh\varnothing eee lll lll ooo$  

Since explicit alignments are not given in the training data, all valid alignments $\pi$ that can generate the target sequence $y$ are summed:

$$P(y | x) = \sum_{\pi \in \mathcal{A}(y, T)} P(\pi | x)$$

where $\mathcal{A}(y, T)$ is the set of all possible alignments of $y$ within $T$ time steps.

---

#### **3. CTC loss function**  
The CTC loss function is defined as the **negative log-likelihood** of the correct transcript $y$ given the input sequence $x$:

$$\mathcal{L}_{CTC} = - \log P(y | x)$$

Since computing this sum over all possible alignments directly is intractable, $P(y | x)$ is computed efficiently using a **dynamic programming algorithm** similar to the **forward-backward algorithm** used in Hidden Markov Models (HMMs).

---

#### **4. Forward-Backward recursion for efficient computation**  
Define $\alpha_t(s)$ as the probability of reaching the first $s$ symbols of the target sequence by time $t$. The recursion formula for the forward variable is:

$$\alpha_t(s) = P(x_t | \pi_t) \cdot (\alpha_{t-1}(s) + \alpha_{t-1}(s-1) + \mathbb{1}_{\pi_s \neq \varnothing} \cdot \alpha_{t-1}(s-2))$$

This efficiently accumulates probabilities over all possible alignments.



*Note:* 
+ The blank token is defined as 0 in the notebook: `blank=0`   
+ A reduction is applied to the loss: `reduction='mean'`.   
By default, PyTorch provides three options for `reduction`:  
  1. **`'none'`** → Returns the loss for each sample **without aggregation**.  
  2. **`'sum'`** → Returns the **sum** of all sample losses.  
  3. **`'mean'`** → Returns the **average** loss, **divided by the total number of targets** in the batch.

When using `reduction='mean'`, PyTorch **normalizes the loss based on the number of phonemes/characters**, making training more stable. As the batch has variable-length sequences, it is usually the **best choice** to avoid bias toward longer or shorter sequences.


Get more information about CTCLoss [here](https://static.googleusercontent.com/media/research.google.com/fr//pubs/archive/43908.pdf) and about the Pytorch CTCLoss [here](https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html). 





<br>

<div class="alert alert-danger">

⚠️ In the ctc loss function, it is mandatory to fill the input and target lenghts **without padding**:

</div>

```python
    input_lengths = torch.tensor([torch.count_nonzero(mfcc, axis=0)[0] for mfcc in mfccs], dtype=torch.long)
    target_lengths = torch.tensor([len(seq[seq < 99]) for seq in target], dtype=torch.long)
```

<br>


<div class="alert alert-danger">

⚠️ Moreover, it is **impossible** to have `input_length < target_length`. This will result in an **error** when calculating the loss, because the CTC loss works by **learning the alignment** between the input (MFCC) and the target (transcription).   
So it must have **at least as many temporal units (`input_length`) as the number of characters (`target_length`)**, otherwise it will not be able to generate the target sequence.   
That is why in the `process_audio_and_text` function, there was this condition:

</div>


```python
    # The number of audio feature frames must be greater than or equal to the number of phoneme labels...
    if (features.shape[0]) < (len(encoded_phonemes)):
        print(f"Skipping entry otherwise CTC will be infinite...")
        return 
    else:
        X.append(features)
        Y.append(encoded_phonemes)
```


<br>

#### Now, it is time for training:

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 

In the paper, the optimizer used is the asynchronous stochastic gradient descent (ASGD):   
> We train the models in a distributed manner using asynchronous stochastic gradient descent (ASGD) optimization technique allowing parallelization of training over a large number of machines on a cluster and enabling large scale training of neural networks.   

In this notebook, as a single machine was used for the training, Adam was chosen.  

In [ ]:
tolerance = 5e-3
max_epochs = 10000
epoch = 0
verbose = True

model.train()
loss_hist = []
batch_size = 1

while True:
    # Reset hidden LSTM states
    hidden = model.init_hidden(batch_size)

    optimizer.zero_grad()
    output = model(input, input_length)
    predictions = torch.argmax(output, dim=-1).squeeze(0).detach().cpu().numpy()


    loss = ctc_loss(output.permute(1, 0, 2), target, input_length, target_length)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    optimizer.step()

    loss_hist.append(loss.item())
    
    epoch += 1

    if verbose:
        print(predictions)
        
    if loss.item() < tolerance or epoch >= max_epochs:
        print(f"\nConvergence reached at epoch {epoch} with a loss of {loss.item():.6f}")
        torch.save(model.state_dict(), "model/single.pth")
        break

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(22, 8))
plt.plot(loss_hist, color='blue', linestyle='-', marker='o')
plt.title('Learning rate evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

In [40]:
model.eval()
output = model(input, input_length)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for i in range(output.shape[1]):
    fig.add_trace(go.Scatter(
        x=list(range(output.shape[2])),
        y=output[0, i, :].detach().numpy(),
        mode='lines',
        name=f'{i}-th timestamp'
    ))

fig.update_layout(
    title='Logits for each timestamp',
    xaxis_title='Phoneme number',
    yaxis_title='Log-probability',
    legend_title='Timestep'
)

fig.show()

This graph shows the probabilities (or more precisely, the log-probabilities) of the module's predictions for each output in the vocabulary.   
There are very sharp peaks, which means that for each timestamp, the model is confident in its predictions.

In [ ]:
prediction = torch.argmax(output, dim=-1)
prediction

In [ ]:
target

In [44]:
def decode_phonemes(encoded_sequence, vocab):
    # Invert the dictionary to obtain an index → phoneme mapping
    inverse_vocab = {v: k for k, v in vocab.items()}

    # Decoding the numerical sequence into phonemes
    decoded_phonemes = [inverse_vocab.get(idx) for idx in encoded_sequence]
    
    return decoded_phonemes

In [ ]:
decoded_phonemes = decode_phonemes(prediction.squeeze(0).detach().numpy(), vocab)
print(f"model's output: {decoded_phonemes}")
print(f"target: {decode_phonemes(target.squeeze(0).detach().numpy(), vocab)}")

After training the model on a **single audio sample** for more than **150 epochs**, the prediction closely matches the target with high accuracy. However, the alignment is not necessarily identical, as the same phoneme can appear across multiple timesteps, leading to some **repetitions** in the output. This behavior is expected due to the nature of sequence modeling in ASR, where **different alignments can still yield the correct transcription**. Despite these variations, the result is highly **satisfying**, demonstrating that the model has successfully learned the mapping from audio features to phonemes.


<div class="alert alert-info" role="alert">

In the following cell, **you can try to train the model with your own vocal**. But please, make sure:
- The vocal is is in english.
- The vocal is "longer" than the transcript:
</div>

  ```python 
    (vocal.shape[0] > len(transcript))
  ```

In [46]:
tolerance = 5e-3
max_epochs = 10000
verbose = True
batch_size = 1

def train_single_audio(audio, transcript):
    # Extract input features
    y, sr = librosa.load(audio, sr=16000)
    features = normalize_mfccs(stack_decimate_frames(extract_features(y)))
    input_tensor = torch.as_tensor(features, dtype=torch.float32).unsqueeze(0)
    input_length = torch.tensor([input_tensor.shape[1]], dtype=torch.long)


    # Encode target
    target = np.array(encode_phonemes(transcript, vocab))
    target_length = torch.tensor([len(target)], dtype=torch.long)
    target = torch.tensor(target, dtype=torch.int32).unsqueeze(0)

    if input_length < target_length:
        raise ValueError("Enable to process this audio!\nTarget must be shorter!")
        return
    
    epoch = 0
    while True:
        hidden = model.init_hidden(batch_size)
        optimizer.zero_grad()

        output = model(input, input_length)
        predictions = torch.argmax(output, dim=-1).squeeze(0).detach().cpu().numpy()


        loss = ctc_loss(output.permute(1, 0, 2), target, input_length, target_length)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        epoch += 1

        if verbose:
            print(predictions)
            
        if loss.item() < tolerance or epoch >= max_epochs:
            print(f"\nConvergence reached at epoch {epoch} with a loss of {loss.item():.6f}")
            return 0

In [47]:
# UNCOMMENT

audio = '....mp3'
transcript = '...'
model = SpeechRecognitionModel(input_dim, hidden_dim, num_layers, num_classes)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
# model.train()

# train_single_audio(audio, transcript)


# model.eval()
# output = model(input, input_length)
# predictions = torch.argmax(output, dim=-1)
# decoded_phonemes = decode_phonemes(predictions.squeeze(0).detach().numpy(), vocab)
# print(f"model's output: {decoded_phonemes}")

#### The next step is to **generalize** the model to handle multiple audio samples instead of just one.  
Training on a single sample allows the model to **overfit** to that specific example, but for real-world ASR, it needs to learn a more **general representation** of speech.  

In [48]:
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create PyTorch datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create PyTorch dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

<div class="alert alert-info" role="alert", align="center">

The following cell is the training cell. You can uncomment and run it if you want, but to avoid losing time, weights of trained models are stored in the `model/` folder.
</div>

In [43]:
from torch.optim.lr_scheduler import ReduceLROnPlateau


num_epochs = 75
batch_size = 8

model = SpeechRecognitionModel(input_dim, hidden_dim, num_layers, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5) 
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
# model.train()

# train_losses = []
# val_losses = []
# lrs = []

# for epoch in range(num_epochs):
#     total_train_loss = 0

#     for i, (X, y) in enumerate(train_dataloader):
#         mfccs, targets = X.to(device), y.to(device)

#         input_length = torch.tensor([torch.count_nonzero(mfcc, axis=0)[0] for mfcc in mfccs], dtype=torch.long).to(device)
#         target_length = torch.tensor([len(seq[seq < 99]) for seq in targets], dtype=torch.long).to(device)

#         hidden = model.init_hidden(batch_size)
#         optimizer.zero_grad()
        
#         outputs = model(mfccs, input_length)

#         loss = ctc_loss(
#             outputs.permute(1, 0, 2), # (T, N, C)
#             targets,
#             input_length,
#             target_length
#         )
#         loss.backward()

#         # Limiting memory cell gradients to [-1, 1].
#         for param in model.lstm.parameters():
#             if param.grad is not None:
#                 param.grad.data.clamp_(-1, 1)

#         optimizer.step()
        
#         total_train_loss += loss.item()
#         print(f"Batch {i + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}")


#     avg_train_loss = total_train_loss / len(train_dataloader)
#     train_losses.append(avg_train_loss)

#     model.eval()
#     total_val_loss = 0
#     with torch.no_grad():
#         for i, (X, y) in enumerate(test_dataloader):
#             mfccs, targets = X.to(device), y.to(device)

#             input_length = torch.tensor([torch.count_nonzero(mfcc, axis=0)[0] for mfcc in mfccs], dtype=torch.long).to(device)
#             target_length = torch.tensor([len(seq[seq < 99]) for seq in targets], dtype=torch.long).to(device)
        
#             outputs = model(mfccs, input_length)

#             loss = ctc_loss(
#                 outputs.permute(1, 0, 2),
#                 targets,
#                 input_length,
#                 target_length
#             )

#             total_val_loss += loss.item()

#     avg_val_loss = total_val_loss / len(test_dataloader)
#     lrs.append(optimizer.param_groups[0]['lr'])
#     scheduler.step(avg_val_loss) 
#     val_losses.append(avg_val_loss)

#     print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

Here is the evolution of the loss for the trained model:
<h3 align='center'>
    <img src="img/loss.png">
</h3>

<br>

And the evolution of its learning rate: 
<h3 align='center'>
    <img src="img/lr.png">
</h3>

<br>

It is noteworthy that the training loss decreases, but the validation loss doesn't really decrease, despite the techniques used to prevent overfitting, such as the scheduler, dropout, and weight decay. Starting from epoch 50, the scheduler reduces the learning rate to 0, and the model no longer improves.

In [44]:
# torch.save(model.state_dict(), "model/model_weights_augmented.pth")

The paper also recommends to clip the gradients of memory cells:
> We clip the activations of memory cells to [-50, 50], and their gradients to [-1, 1], making CTC training stable.

That is why in the training loop there is this code to limit the norm of the gradients:
```python
    loss.backward()

    # Limiting memory cell gradients to [-1, 1].
    for param in model.lstm.parameters():
        if param.grad is not None:
            param.grad.data.clamp_(-1, 1)

    optimizer.step()
```

<br>

#### Now, it is time to test the model:

In [50]:
path_to_model = "model/model_weights_augmented.pth"

In [ ]:
# Load the pre-trained model
saved_model = SpeechRecognitionModel(input_dim, hidden_dim, num_layers, num_classes)
saved_model.load_state_dict(torch.load(path_to_model, weights_only=True))
saved_model.eval()

The following audio and transcript will be used for the test:

In [52]:
audio_test = 'data/audio_sample/common_voice_en_38614407.mp3' 
transcript = "I'm in excellent health."

In [53]:
y, sr = librosa.load(audio_test, sr=16000)
features = normalize_mfccs(stack_decimate_frames(extract_features(y)))
input = torch.as_tensor(features, dtype=torch.float32).unsqueeze(0)
input_length = torch.tensor([input.shape[1]], dtype=torch.long)

encoded_phonemes = encode_phonemes(transcript, vocab)
target = np.array(encoded_phonemes)

In [ ]:
prediction = saved_model(input, input_length)
prediction

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for i in range(prediction.shape[1]):
    fig.add_trace(go.Scatter(
        x=list(range(prediction.shape[2])),
        y=prediction[0, i, :].detach().numpy(),
        mode='lines',
        name=f'{i}-th timestamp'
    ))

fig.update_layout(
    title='Logits for each timestamp',
    xaxis_title='Phoneme number',
    yaxis_title='Log-probability',
    legend_title='Timestep'
)

fig.show()

On the graph, the peaks are much less sharp than when the model was trained on a single audio. It indicates **high uncertainty** in the model's prediction. Ideally, a well-trained ASR model should assign a high probability to a single phoneme (or blank token in CTC) at each timestep.  

In [56]:
encoded_prediction = torch.argmax(prediction, dim=-1)

In [ ]:
decoded_prediction = decode_phonemes(encoded_prediction.flatten().detach().cpu().numpy(), vocab)

print(f"Encoded predictions: {encoded_prediction}")
print(f"Decoded predictions: {decoded_prediction}")


decoded_target_phonemes = decode_phonemes(target, vocab)
print(f"Encoded target phonemes: {target}")
print(f"Decoded target phonemes: {decoded_target_phonemes}")

The prediction is very mediocre and this is because, the modele is mediocre. It still has trouble generalizing despite the addition of `dropout`, `weight decay`, and a `scheluder` that adjusts the learning rate because:

- Insufficient or non-representative data:
The dataset is probably not **big** enough, the model can **learn specific patterns** instead of generalizing. But the current dataset contains 3000 audios and it took already a lot of time to process.

- Use of LSTMs
    Today Transformers have replaced LSTMs in Speech Recognition. Here's why:

    | **Criterion** | **LSTMs** | **Transformers** |
    |--------------------|--------------------------------|--------------------------------|
    | **Long-term dependencies** | Memory problem on long sequences (vanishing gradient) | Learns on the entire sequence at once thanks to **attention** |
    | **Parallelization** | Sequential → slow to train | Can train **much faster** thanks to parallelism |
    | **Captures global relations** | Depends only on previous hidden states | Captures **global** relations between phonemes and words |
    | **Adapts to noisy data** | Sensitive to noise (because it depends too much on the past) | More robust thanks to **self-attention layers** |
    | **Performance on large datasets** | Can saturate when too much data | Improves with more data (scalability) |

    Concretely:
    1. **LSTMs are less efficient at learning long dependencies**:   
        In speech recognition, some relationships **between phonemes are distant**, so an LSTM may have trouble capturing this information well.

    2. **Transformers (Ex: Wav2Vec2, Conformer) are better for Speech-to-Text**:  
        Today, **almost all state-of-the-art models** in speech recognition are based on Transformers (ex: **[Whisper](https://openai.com/index/whisper/), [Wav2Vec2](https://ai.meta.com/research/impact/wav2vec/), [Conformer](https://research.google/pubs/conformer-convolution-augmented-transformer-for-speech-recognition/)**). 

    3. **LSTMs are sequential**:   
        Training cannot be parallelized efficiently, so **Transformers learn faster** with more data.


<br>

The paper was realized by a *Google* team, which has obviously better datasets, better infrastructure and computing power to train a model for a long time.   


<br>

#### For the last part of the notebook, the model that learned the sentence "Jesus loves me, he who died" perfectly will be used:

In [58]:
path_to_model = "model/single.pth"
saved_model = SpeechRecognitionModel(input_dim, hidden_dim, num_layers, num_classes)
saved_model.load_state_dict(torch.load(path_to_model, weights_only=True))
saved_model.eval()


audio_test = 'data/audio_sample/common_voice_en_39177958.mp3' 
transcript = 'Jesus loves me, he who died.'

# Extract input features
y, sr = librosa.load(audio_test, sr=16000)
features = normalize_mfccs(stack_decimate_frames(extract_features(y)))
input = torch.as_tensor(features, dtype=torch.float32).unsqueeze(0)
input_length = torch.tensor([input.shape[1]], dtype=torch.long)


# Encode target
target = np.array(encode_phonemes(transcript, vocab))
target_length = torch.tensor([len(target)], dtype=torch.long)
target = torch.tensor(target, dtype=torch.int32).unsqueeze(0)
output = saved_model(input, input_length)
prediction = torch.argmax(output, dim=-1).squeeze(0)
decoded_prediction = decode_phonemes(prediction.detach().cpu().numpy(), vocab)

The objective has almost been achieved, all that's left to do is to convert the phoneme sequences into text.


## IV. From phonemes to text


Once the model generates predictions and decodes them into phonemes, the next challenge is converting the sequence of phonemes into human-readable text. This process involves **decoding** the phoneme sequence into actual words. This task is not straightforward, as prediction errors are possible. Therefore, it is crucial to decode a coherent and contextually accurate transcription from the sequence of phonemes. Several techniques can be employed to achieve this:

- **greedy decoding**, where the most likely phoneme is chosen at each timestep, resulting in a straightforward, but potentially suboptimal, transcription. While this method is simple, it may not always handle ambiguities or multiple possible transcriptions well. 
- **beam search decoding**, which explores multiple possible sequences of phonemes by maintaining a "beam" of the top \( k \) most probable sequences at each timestep. This approach allows for better handling of uncertainty in the model's predictions, ultimately producing more accurate transcriptions.
- **language models**: Some systems use LM in conjunction with the phoneme decoder to improve performance. A language model helps to filter out improbable phoneme sequences by assigning higher probabilities to sequences that conform to natural language rules. This post-processing step can be particularly useful when the phoneme sequence alone might lead to nonsensical or ambiguous outputs.
- **CTC decoding**: These methods are often used in speech recognition systems, especially in the context of models trained with Connectionist Temporal Classification (CTC). In this case, the CTC loss function itself implicitly helps the model find the best alignment between audio and transcript, but a dedicated decoding algorithm like **CTC beam search** is needed to handle the blank symbols and to find the optimal transcription.

For more information about context dependency, here is a [paper](https://static.googleusercontent.com/media/research.google.com/fr//pubs/archive/43910.pdf).

In the evaluation of ASR (Automatic Speech Recognition) models, one of the most commonly used metrics is the **Word Error Rate (WER)**. This metric provides an indication of the accuracy of the model's transcriptions by comparing the predicted text to the reference transcription.

The **WER** is calculated based on the number of insertions, deletions, and substitutions required to transform the predicted transcript into the correct reference transcript. Specifically, it is computed as:

$$WER = \frac{S + D + I}{N}$$

where:
- $S$: the number of substitutions (incorrect words).
- $D$: the number of deletions (missing words).
- $I$: the number of insertions (extra words). 
- $N$: the total number of words in the reference transcript.

A **lower WER** indicates better performance, as it reflects fewer errors in the predicted transcription. The metric is widely used because it directly reflects the quality of transcriptions, and it is simple to compute. However, while WER is useful for quantifying overall accuracy, it does not always capture the nuances of word context or phonetic correctness, which can be important in certain applications of ASR systems.

In [59]:
def compute_wer(target_transcript, predicted_transcript):
    # Tokenize transcripts into words
    target_words = target_transcript.split()
    predicted_words = predicted_transcript.split()

    len_target = len(target_words)
    len_predicted = len(predicted_words)

    # Handle edge case: Empty target transcript
    if len_target == 0:
        return 1.0 if len_predicted > 0 else 0.0  # If target is empty but prediction isn't, WER = 100%

    # Initialize DP matrix
    dp = [[0] * (len_predicted + 1) for _ in range(len_target + 1)]

    # Fill base cases
    for i in range(len_target + 1):
        dp[i][0] = i  # Deletions
    for j in range(len_predicted + 1):
        dp[0][j] = j  # Insertions

    # Compute WER using dynamic programming
    for i in range(1, len_target + 1):
        for j in range(1, len_predicted + 1):
            if target_words[i - 1] == predicted_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # No operation needed
            else:
                dp[i][j] = min(dp[i - 1][j - 1],  # Substitution
                               dp[i - 1][j],      # Deletion
                               dp[i][j - 1]) + 1  # Insertion

    # Compute final WER
    wer = dp[len_target][len_predicted] / len_target
    return wer


### 1. Greedy decoder
Greedy search is the simplest decoding method. It selects the word with the highest probability as its next word: $w_t=argmax_w P(w∣w_{1:t−1})$ at each timestep $t$. The following sketch shows greedy search.






An greedy algorithm based on **CMUdict (Carnegie Mellon Pronouncing Dictionary)**, was designed. The [CMUdict](https://en.wikipedia.org/wiki/CMU_Pronouncing_Dictionary) is a **phonetic dictionary** for **American English**, developed by the **Carnegie Mellon University Speech Group**. It provides **word-to-phoneme mappings**, where each word is transcribed in terms of its **phonemes using the ARPAbet notation**.  

In [ ]:
import nltk
from nltk.corpus import cmudict
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('cmudict')

# Download the CMU Pronouncing Dictionary
cmu_dict = cmudict.dict()

# Inverse dictionary {tuple(phonemes) -> word}
phoneme_to_word = {}
for word, pron_list in cmu_dict.items():
    clean_pron = tuple(p.rstrip('012') for p in pron_list[0])
    phoneme_to_word[clean_pron] = word.lower()

def phonemes_to_text_greedy(phonemes):

    phonemes_clean = [char for char in phonemes if char != '-'] 

    result_words = []
    start = 0

    while start < len(phonemes_clean):
        found = False
        # Try to find the longest word possible
        for end in range(len(phonemes_clean), start, -1):
            sub_seq = tuple(phonemes_clean[start:end])
            if sub_seq in phoneme_to_word:
                result_words.append(phoneme_to_word[sub_seq])
                start = end  # Move on to the next phonemes
                found = True
                break

        if not found:
            start += 1  # Go forward one phoneme to try again

    return " ".join(result_words)

In [ ]:
cmu_dict

First, it is necessary to process the prediction. in fact, as the model can provide multiple times the same output if the phoneme stretches on multiple timestamps. Thus, consecutive equal phonemes must be deleted:

In [62]:
def remove_consecutive_duplicates(lst):
    result = []
    for i in range(len(lst)):
        if i == 0 or lst[i] != lst[i-1]:
            result.append(lst[i])
    return result

In [ ]:
decoded_prediction_processed = remove_consecutive_duplicates(decoded_prediction)
print(f"Model ouput: {decoded_prediction_processed}")
greedy_output = phonemes_to_text_greedy(decoded_prediction_processed)
print(f"Greedy algorithm ouput: {greedy_output}")
print(f"Target: {transcript}")
print(f"WER: {compute_wer(transcript, greedy_output)}")

The WER here is equal to 1, because there is an error for every word. The greedy decoder is pretty mediocre!

<br>


### 2. Beam search decoder   
Unlike a greedy decoder, which selects the most probable phoneme sequence at each timestep, **beam search** considers multiple possible sequences and keeps the most probable ones. This helps **correct errors** and **produce more coherent words**.   

**Beam width** (also called **beam size**) is a key **hyperparameter** in **beam search decoding**. It determines **how many candidate sequences** are kept at each step: 
- At each step, the decoder **keeps only the top-k most probable sequences**.
- A larger beam width allows the decoder to explore **more possibilities**, improving accuracy but increasing computational cost.
(If beam width = 1, the decoder chooses the most probable word at each step → Greedy decoding (fast but often inaccurate))

A beam search takes much longer than a greedy search but is much more accurate. This is why beam search is not used in ASR systems, where a rapid response is essential. Instead, language models (LMs) are used.

<br>

### 3. LM

[KenLM](https://github.com/kpu/kenlm) is a **fast and efficient n-gram language model** widely used in speech recognition and NLP tasks. It helps in decoding by ranking possible word sequences based on their probability in natural language.